# MessagesPlaceholder 
- In LangChain, MessagesPlaceholder is used to allow the use of placeholders within a conversation flow or prompt. These placeholders can later be dynamically filled with values at runtime, making the conversation more flexible and context-aware

In [1]:
#Example without MessagesPlaceholder
from langchain.prompts import PromptTemplate

template = "Hello, my name is {name}. How can I help you today?"
prompt = PromptTemplate(input_variables=["name"], template=template)
final_prompt = prompt.format(name="John")
print(final_prompt)


Hello, my name is John. How can I help you today?


In [ ]:
from langchain.agents import initialize_agent, Tool, AgentType
from langchain.agents import create_openai_functions_agent
from langchain.llms import OpenAI

# Initialize the language model (OpenAI GPT)
llm = OpenAI(temperature=0)


# Create some sample tools (you can replace these with actual tools like APIs or databases)
tools = [
    
    Tool(
        name="multiply",
        func=multiply,
        description="Invoke this method only when you have two inputs in query and multiply or product of two numbers and return"
    ),
    Tool(
        name="sqrt",
        func=sqrt,
        description="Invoke this method only when you have one positive number input in query do the square root of it and return"
    )
  
]


# Initialize the zero-shot agent
agent = initialize_agent(
    tools=tools,
    agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
    llm=llm,
    verbose=True
)


# Example task: Using the zero-shot agent to handle a question
query = "Tell me the square root of number 625"

response = agent.run(query)

print(response)



# Example task: Using the zero-shot agent to handle a question
query = "Please tell previous query"

response = agent.run(query)

print(response)






> Entering new AgentExecutor chain...
 I should use the sqrt tool to find the square root of 625
Action: sqrt
Action Input: 625
Observation: 25.0
Thought: I now know the final answer
Final Answer: 25.0

> Finished chain.
25.0


> Entering new AgentExecutor chain...
 I need to access the previous query to get the input
Action: multiply
Action Input: previous query
Observation: 
Thought: I need to access the previous query to get the input
Action: sqrt
Action Input: previous query

ValidationError: 1 validation error for sqrt
number
  Input should be a valid number, unable to parse string as a number [type=float_parsing, input_value='previous query', input_type=str]
    For further information visit https://errors.pydantic.dev/2.10/v/float_parsing

In [9]:


prompt = MessagesPlaceholder("history", optional=True)

prompt.format_messages() # returns empty list []

prompt.format_messages(
    history=[
        ("system", "You are an AI assistant."),
        ("human", "Hello!"),
    ]
)

[SystemMessage(content='You are an AI assistant.', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hello!', additional_kwargs={}, response_metadata={})]

In [10]:
#Building a prompt with chat history

from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful assistant."),
        MessagesPlaceholder("history"),
        ("human", "{question}")
    ]
)
prompt.invoke(
   {
       "history": [("human", "what's 5 + 2"), ("ai", "5 + 2 is 7")],
       "question": "now multiply that by 4"
   }
)



ChatPromptValue(messages=[SystemMessage(content='You are a helpful assistant.', additional_kwargs={}, response_metadata={}), HumanMessage(content="what's 5 + 2", additional_kwargs={}, response_metadata={}), AIMessage(content='5 + 2 is 7', additional_kwargs={}, response_metadata={}), HumanMessage(content='now multiply that by 4', additional_kwargs={}, response_metadata={})])

In [11]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq

load_dotenv()


groq_api_key=os.getenv('GROQ_API_KEY')

groq_model = ChatGroq(model="gemma2-9b-it",groq_api_key=groq_api_key)

In [12]:
import bs4
from langchain import hub
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_community.vectorstores import FAISS
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.prompts import ChatPromptTemplate
from langchain_huggingface.embeddings import *
from langchain_text_splitters import RecursiveCharacterTextSplitter

USER_AGENT environment variable not set, consider setting it to identify your requests.


In [27]:
# 1. Load, chunk and index the contents of the blog to create a retriever.
loader = WebBaseLoader(
    web_paths=("https://lilianweng.github.io/posts/2023-06-23-agent/",),
    bs_kwargs=dict(
        parse_only=bs4.SoupStrainer(
            class_=("post-content", "post-title", "post-header")
        )
    ),
)
docs = loader.load()
print(docs)

hugging_face_embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")


[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='\n\n      LLM Powered Autonomous Agents\n    \nDate: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng\n\n\nBuilding agents with LLM (large language model) as its core controller is a cool concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating well-written copies, stories, essays and programs; it can be framed as a powerful general problem solver.\nAgent System Overview#\nIn a LLM-powered autonomous agent system, LLM functions as the agent’s brain, complemented by several key components:\n\nPlanning\n\nSubgoal and decomposition: The agent breaks down large tasks into smaller, manageable subgoals, enabling efficient handling of complex tasks.\nReflection and refinement: The agent can do self-criticism and self-reflection over past actions, learn from mistake

In [29]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=20)
splits = text_splitter.split_documents(docs)
vectorstore = FAISS.from_documents(documents=splits, embedding=hugging_face_embeddings)
retriever = vectorstore.as_retriever()
print(splits)

[Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='LLM Powered Autonomous Agents'), Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Date: June 23, 2023  |  Estimated Reading Time: 31 min  |  Author: Lilian Weng'), Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='Building agents with LLM (large language model) as its core controller is a cool concept. Several'), Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='concept. Several proof-of-concepts demos, such as AutoGPT, GPT-Engineer and BabyAGI, serve as'), Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='BabyAGI, serve as inspiring examples. The potentiality of LLM extends beyond generating'), Document(metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_cont

In [30]:
system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)

question_answer_chain = create_stuff_documents_chain(groq_model, prompt)

rag_chain = create_retrieval_chain(retriever, question_answer_chain)

In [31]:
response = rag_chain.invoke({"input": "What is Task Decomposition?"})
response["answer"]

'Task decomposition is the process of breaking down large tasks into smaller, more manageable subgoals. This makes it easier for agents to plan and execute complex tasks. LLMs can be prompted to perform task decomposition.  \n'

In [32]:
response = rag_chain.invoke({"input": "What are the benifit to indian cricket team"})
response["answer"]

'This document does not contain information about the benefits to the Indian cricket team.  I need more context to answer your question.  Please provide additional information about Indian cricket. \n'

In [34]:
response = rag_chain.invoke({"input": "What is the last question asked"})
response["answer"]

"I don't know the last question asked. \n"

In [36]:
#Adding chat history

from langchain.chains import create_history_aware_retriever
from langchain_core.prompts import MessagesPlaceholder

contextualize_q_system_prompt = (
    "Given a chat history and the latest user question "
    "which might reference context in the chat history, "
    "formulate a standalone question which can be understood "
    "without the chat history. Do NOT answer the question, "
    "just reformulate it if needed and otherwise return it as is."
)

contextualize_q_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", contextualize_q_system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)
history_aware_retriever = create_history_aware_retriever(
    groq_model, retriever, contextualize_q_prompt
)



In [ ]:
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain

qa_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder("chat_history"),
        ("human", "{input}"),
    ]
)


question_answer_chain = create_stuff_documents_chain(groq_model, qa_prompt)

rag_chain = create_retrieval_chain(history_aware_retriever, question_answer_chain)

In [41]:
from langchain_core.messages import AIMessage, HumanMessage

chat_history = []

question = "What is Task Decomposition?"
ai_msg_1 = rag_chain.invoke({"input": question, "chat_history": chat_history})
chat_history.extend(
    [
        HumanMessage(content=question),
        AIMessage(content=ai_msg_1["answer"]),
    ]
)

second_question = "What is the last question asked"
ai_msg_2 = rag_chain.invoke({"input": second_question, "chat_history": chat_history})

print(ai_msg_2["answer"])

The last question asked was: "What is Task Decomposition?" 



In [ ]:
#meTHOd-2 USING SESSION 

In [49]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory


store = {}


def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in store:
        store[session_id] = ChatMessageHistory()
    return store[session_id]


with_message_history = RunnableWithMessageHistory(rag_chain,
                                                  get_session_history,
                                                  input_messages_key="input",
                                                  history_messages_key="chat_history",
                                                  output_messages_key="answer",
                                                   )
config = {"configurable": {"session_id": "abc2"}}

with_message_history.invoke({"input": "What is Task Decomposition?"}, config=config)

with_message_history.invoke({"input": "What is 2+2"}, config=config)

with_message_history.invoke({"input": "What is last question"}, config=config)





{'input': 'What is last question',
 'chat_history': [HumanMessage(content='What is Task Decomposition?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Task decomposition is the process of breaking down large tasks into smaller, more manageable subgoals.  \n\nThis makes tasks easier to plan and execute. LLMs can be used for task decomposition with simple prompting techniques. \n', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What is 2+2', additional_kwargs={}, response_metadata={}),
  AIMessage(content='2 + 2 = 4. \n', additional_kwargs={}, response_metadata={})],
 'context': [Document(id='182ef608-eed3-4216-804f-9489deec8f2b', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='2. Constructively self-criticize your big-picture behavior constantly.'),
  Document(id='f057871c-b671-4b09-906c-5cf007eb2885', metadata={'source': 'https://lilianweng.github.io/posts/2023-06-23-agent/'}, page_content='is, in the 